## Demo for SDK (Lite Version)

This example shows how to use SDK to deploy a task using a pre-defined repo source.

- [Getting Started](#Getting-Started)
- [Creating And Deploying Task](#Creating-And-Deploying-Task)
    - [Create task](#create-task)
- [View Deployed Task Results](#View-Deployed-Task-Results)
    - [Follow up task status](#follow-up-task-status-optional)
    - [Show result](#show-result)
- [Update Task](#update-task-optional)
    - [Renew task](#renew-task-optional)
    - [Terminate task](#terminate-task-optional)

# Getting Started


### Initialization

It is recommended to use `python-dotenv` and add environment file `.env` in your local directory.

For example, in this demo notebook the following env variables are used:

```
API_KEY=
WALLET=
PK=
```

In [ ]:
import sys
sys.path.insert(0, '..') 

import os
import time
import dotenv
import json
dotenv.load_dotenv()

import swan

wallet_address = os.getenv('WALLET')
private_key = os.getenv('PK')
api_key = os.getenv('API_KEY')

# for testnet prod (proxima)
orchestrator = swan.resource(
    api_key=api_key, 
    service_name='Orchestrator'
)

# # for mainnet
# orchestrator = swan.resource(
#     api_key=api_key, 
#     service_name='Orchestrator',
#     network='mainnet'
# )


## Creating And Deploying Task

Show repository image of pre-defined applications (optional)

### Create task

Choose a application repository image name from the above list, and provide wallet address and private key to `create_task` interface to deploy a task.

In [ ]:
result = orchestrator.create_task(
    wallet_address=wallet_address,
    app_repo_image='Tetris', #'hello_world',
    private_key=private_key
)

# print(json.dumps(result, indent=2))
task_uuid = result.get('id')
tx_hash = result.get('tx_hash')

If completed steps above, please go to [View Deployed Task Results](#View-Deployed-Task-Results) to view results, or [terminate task](#terminate-task) for more functions related to tasks

## View Deployed Task Results

### follow up task status (optional)
The following step is optional, shows information when waiting for task being deployed.

In [ ]:
# Check task info
info = orchestrator.get_deployment_info(task_uuid=task_uuid)
print(json.dumps(info, indent=2))

### Show result

`job_real_uri` is for show the result of application you deployed. Click the url to view application in browser.

In [ ]:
result_url = orchestrator.get_real_url(task_uuid)
print(result_url)

## Update Task (optional)

### Renew task (optional)

Extend `task_uuid` by `duration`. Using auto pay automatically makes a transaction to SWAN contract and extends the task.

In [ ]:
renew_task = orchestrator.renew_task(
    task_uuid=task_uuid, 
    duration=66, # Optional: Defaults to 3600 seconds (1 hour)
    auto_pay=True, # Optional: Defaults to False, in this demo path set to True
    private_key=private_key
)

print(json.dumps(renew_task, indent=2))
tx_hash = renew_task.get('tx_hash')

### Terminate task (Optional)

Terminate the task `task_uuid` and get a refund for remaining time

In [ ]:
terminate_status = orchestrator.terminate_task(task_uuid)
if terminate_status['status'] == 'success':
    print(f"Terminated {task_uuid} successfully")
else:
    print(f"Failed to terminate {task_uuid}")

### Config Order Status (Optional)

Check config order status with `task_uuid` and `tx_hash`, for example, when create a task, a config order of type `Creation` is created in database with the payment information if available; when renew a task, a `Renewal` config order is created.

We can check the status of these request to see if the payment has been validated and config order has been executed.

In [ ]:
res = orchestrator.get_config_order_status(task_uuid=task_uuid, tx_hash=tx_hash)
print(json.dumps(res, indent=2))